In [1]:
# Cell 1: Imports, Configuration & Logging Setup
import os, sys
import logging
import argparse
import torch, torch.nn as nn
import torchvision.transforms as T
from PIL import Image
import cv2
import numpy as np
import librosa
import json
import time
from datetime import datetime
import traceback
from pathlib import Path

# Configure logging
def setup_logging(log_level="INFO", log_file=None):
    """Setup comprehensive logging configuration"""
    # Create logs directory
    log_dir = "/kaggle/working/logs"
    os.makedirs(log_dir, exist_ok=True)
    
    # Default log file with timestamp
    if log_file is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = os.path.join(log_dir, f"deepfake_inference_{timestamp}.log")
    
    # Configure logging format
    log_format = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s'
    )
    
    # Setup file handler
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(log_format)
    
    # Setup console handler
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(getattr(logging, log_level))
    console_format = logging.Formatter('%(levelname)s - %(message)s')
    console_handler.setFormatter(console_format)
    
    # Configure root logger
    root_logger = logging.getLogger()
    root_logger.setLevel(logging.DEBUG)
    root_logger.handlers = []  # Clear existing handlers
    root_logger.addHandler(file_handler)
    root_logger.addHandler(console_handler)
    
    return log_file

# Initialize logging
LOG_FILE = setup_logging("INFO")
logger = logging.getLogger(__name__)

# Configuration
CONFIG = {
    "models_dir": "/kaggle/input/all-models-trained",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "img_size": (224, 224),
    "audio_sr": 16000,
    "audio_n_mels": 128,
    "audio_duration": 2.5,
    "video_max_frames": 8,
    "batch_size": 32,
    "inference_timeout": 30  # seconds
}

logger.info("="*60)
logger.info("DEEPFAKE DETECTION INFERENCE PIPELINE INITIALIZED")
logger.info("="*60)
logger.info(f"Device: {CONFIG['device']}")
logger.info(f"Models directory: {CONFIG['models_dir']}")
logger.info(f"Log file: {LOG_FILE}")

# Verify GPU availability
if torch.cuda.is_available():
    gpu_info = torch.cuda.get_device_properties(0)
    logger.info(f"GPU: {gpu_info.name}, Memory: {gpu_info.total_memory // 1024**3} GB")
else:
    logger.warning("GPU not available, using CPU (inference will be slower)")

# Common transforms
IMAGE_TRANSFORM = T.Compose([
    T.Resize(CONFIG["img_size"]),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

VIDEO_TRANSFORM = IMAGE_TRANSFORM  # Apply per-frame
logger.info("Transform pipelines initialized")


INFO - ============================================================
INFO - DEEPFAKE DETECTION INFERENCE PIPELINE INITIALIZED
INFO - ============================================================
INFO - Device: cuda
INFO - Models directory: /kaggle/input/all-models-trained
INFO - Log file: /kaggle/working/logs/deepfake_inference_20250720_093546.log
INFO - GPU: Tesla T4, Memory: 14 GB
INFO - Transform pipelines initialized


In [2]:
# Cell 2: Enhanced Model Definitions with Logging
def safe_efficientnet(pretrained=False):
    """Load EfficientNet with fallback and logging"""
    try:
        logger.debug(f"Loading EfficientNet with pretrained={pretrained}")
        model = torch.hub.load('pytorch/vision:v0.13.1', 'efficientnet_b0', 
                              pretrained=pretrained, verbose=False)
        logger.debug("EfficientNet loaded successfully")
        return model
    except Exception as e:
        logger.error(f"Failed to load EfficientNet: {e}")
        logger.warning("Falling back to random initialization")
        return torch.hub.load('pytorch/vision:v0.13.1', 'efficientnet_b0', 
                             pretrained=False, verbose=False)

class ImageModel(nn.Module):
    """Image deepfake detection model"""
    def __init__(self):
        super().__init__()
        logger.debug("Initializing ImageModel")
        m = safe_efficientnet(False)
        m.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(1280, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 2)
        )
        self.net = m
        logger.debug("ImageModel initialized successfully")
    
    def forward(self, x):
        return self.net(x)

class VideoModel(nn.Module):
    """Video deepfake detection model"""
    def __init__(self):
        super().__init__()
        logger.debug("Initializing VideoModel")
        m = safe_efficientnet(False)
        m.classifier = nn.Identity()
        self.back = m
        self.lstm = nn.LSTM(1280, 256, 2, batch_first=True, dropout=0.3)
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)
        )
        logger.debug("VideoModel initialized successfully")
    
    def forward(self, x):
        b, f, c, h, w = x.shape
        logger.debug(f"Video input shape: {x.shape}")
        feats = self.back(x.view(b*f, c, h, w)).view(b, f, -1)
        _, (h_n, _) = self.lstm(feats)
        return self.fc(h_n[-1])

class AudioModel(nn.Module):
    """Audio deepfake detection model"""
    def __init__(self):
        super().__init__()
        logger.debug("Initializing AudioModel")
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2), nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2), nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.AdaptiveAvgPool2d((1, 1)), nn.BatchNorm2d(128)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(64, 2)
        )
        logger.debug("AudioModel initialized successfully")
    
    def forward(self, x):
        logger.debug(f"Audio input shape: {x.shape}")
        return self.fc(self.cnn(x))

logger.info("Model classes defined successfully")


INFO - Model classes defined successfully


In [3]:
# Cell 3: Enhanced Preprocessing with Logging
def extract_frames(video_path, num_frames=None):
    """Extract frames from video with comprehensive logging"""
    if num_frames is None:
        num_frames = CONFIG["video_max_frames"]
    
    logger.info(f"Extracting {num_frames} frames from video: {video_path}")
    start_time = time.time()
    
    try:
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video file: {video_path}")
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        duration = total_frames / fps if fps > 0 else 0
        
        logger.debug(f"Video info - Total frames: {total_frames}, FPS: {fps:.2f}, Duration: {duration:.2f}s")
        
        if total_frames == 0:
            raise ValueError("Video contains no frames")
        
        # Calculate frame indices
        if total_frames <= num_frames:
            idxs = list(range(total_frames))
        else:
            idxs = np.linspace(0, total_frames-1, num_frames, dtype=int)
        
        frames = []
        frames_extracted = 0
        
        for i in idxs:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                logger.warning(f"Failed to read frame at index {i}")
                continue
            
            # Convert BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(frame))
            frames_extracted += 1
        
        cap.release()
        
        processing_time = time.time() - start_time
        logger.info(f"Frame extraction completed: {frames_extracted}/{len(idxs)} frames in {processing_time:.2f}s")
        
        if not frames:
            raise ValueError("No frames could be extracted from video")
        
        return frames
        
    except Exception as e:
        logger.error(f"Frame extraction failed: {str(e)}")
        logger.debug(traceback.format_exc())
        raise

def audio_to_mel(audio_path):
    """Convert audio to mel-spectrogram with comprehensive logging"""
    logger.info(f"Converting audio to mel-spectrogram: {audio_path}")
    start_time = time.time()
    
    try:
        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio file not found: {audio_path}")
        
        # Load audio
        y, sr = librosa.load(audio_path, sr=CONFIG["audio_sr"], 
                           duration=CONFIG["audio_duration"])
        logger.debug(f"Loaded audio: shape={y.shape}, sr={sr}")
        
        # Ensure consistent length
        target_length = int(sr * CONFIG["audio_duration"])
        if len(y) > target_length:
            y = y[:target_length]
            logger.debug(f"Truncated audio to {target_length} samples")
        else:
            y = np.pad(y, (0, target_length - len(y)), 'constant')
            logger.debug(f"Padded audio to {target_length} samples")
        
        # Generate mel-spectrogram
        S = librosa.feature.melspectrogram(
            y=y, sr=sr, n_mels=CONFIG["audio_n_mels"],
            n_fft=2048, hop_length=512, power=2.0
        )
        S_db = librosa.power_to_db(S, ref=np.max)
        
        processing_time = time.time() - start_time
        logger.info(f"Mel-spectrogram generated: shape={S_db.shape} in {processing_time:.3f}s")
        
        return S_db
        
    except Exception as e:
        logger.error(f"Audio preprocessing failed: {str(e)}")
        logger.debug(traceback.format_exc())
        raise

def validate_input_file(file_path, expected_extensions):
    """Validate input file with logging"""
    logger.debug(f"Validating input file: {file_path}")
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Input file not found: {file_path}")
    
    file_ext = Path(file_path).suffix.lower()
    if file_ext not in expected_extensions:
        raise ValueError(f"Unsupported file type: {file_ext}. Expected: {expected_extensions}")
    
    file_size = os.path.getsize(file_path)
    logger.info(f"Input file validated: {file_path} ({file_size} bytes, {file_ext})")
    
    return True

logger.info("Preprocessing functions defined successfully")


INFO - Preprocessing functions defined successfully


In [4]:
# Cell 4: Enhanced Inference Functions with Logging
def load_model(model_path, model_type):
    logger.info(f"Loading {model_type} model from: {model_path}")
    start_time = time.time()
    checkpoint = torch.load(model_path, map_location=CONFIG["device"])
    
    # Initialize the correct model
    if model_type == "image":
        model = ImageModel()
    elif model_type == "video":
        model = VideoModel()
    elif model_type == "audio":
        model = AudioModel()
    else:
        raise ValueError(f"Unsupported model type: {model_type}")
    
    # Load state dict with non-strict mode
    state_dict = checkpoint.get('model_state_dict', checkpoint)
    model.load_state_dict(state_dict, strict=False)
    
    model.to(CONFIG["device"]).eval()
    load_time = time.time() - start_time
    logger.info(f"Loaded {model_type} model in {load_time:.2f}s (strict=False)")
    return model


def infer_image(model, image_path):
    """Image inference with comprehensive logging"""
    logger.info(f"Starting image inference: {image_path}")
    
    try:
        # Validate input
        validate_input_file(image_path, ['.jpg', '.jpeg', '.png', '.bmp', '.tiff'])
        
        start_time = time.time()
        
        # Load and preprocess image
        img = Image.open(image_path).convert("RGB")
        logger.debug(f"Image loaded: {img.size}")
        
        x = IMAGE_TRANSFORM(img).unsqueeze(0).to(CONFIG["device"])
        logger.debug(f"Image preprocessed: {x.shape}")
        
        # Run inference
        model.eval()
        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits, 1).cpu().numpy()[0]
        
        processing_time = time.time() - start_time
        
        result = {
            'file_path': image_path,
            'modality': 'image',
            'probabilities': {
                'real': float(probs[0]),
                'fake': float(probs[1])
            },
            'prediction': 'fake' if probs[1] > probs[0] else 'real',
            'confidence': float(max(probs)),
            'processing_time': processing_time,
            'timestamp': datetime.now().isoformat()
        }
        
        logger.info(f"Image inference completed: {result['prediction']} "
                   f"({result['confidence']:.3f} confidence) in {processing_time:.3f}s")
        
        return result
        
    except Exception as e:
        logger.error(f"Image inference failed: {str(e)}")
        logger.debug(traceback.format_exc())
        return {
            'file_path': image_path,
            'modality': 'image',
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        }

def infer_video(model, video_path):
    """Video inference with comprehensive logging"""
    logger.info(f"Starting video inference: {video_path}")
    
    try:
        # Validate input
        validate_input_file(video_path, ['.mp4', '.avi', '.mov', '.mkv', '.webm'])
        
        start_time = time.time()
        
        # Extract frames
        frames = extract_frames(video_path)
        if not frames:
            raise ValueError("No frames extracted from video")
        
        logger.debug(f"Processing {len(frames)} frames")
        
        # Preprocess frames
        batch = torch.stack([VIDEO_TRANSFORM(f) for f in frames]).unsqueeze(0).to(CONFIG["device"])
        logger.debug(f"Video batch shape: {batch.shape}")
        
        # Run inference
        model.eval()
        with torch.no_grad():
            logits = model(batch)
            probs = torch.softmax(logits, 1).cpu().numpy()[0]
        
        processing_time = time.time() - start_time
        
        result = {
            'file_path': video_path,
            'modality': 'video',
            'frames_processed': len(frames),
            'probabilities': {
                'real': float(probs[0]),
                'fake': float(probs[1])
            },
            'prediction': 'fake' if probs[1] > probs[0] else 'real',
            'confidence': float(max(probs)),
            'processing_time': processing_time,
            'timestamp': datetime.now().isoformat()
        }
        
        logger.info(f"Video inference completed: {result['prediction']} "
                   f"({result['confidence']:.3f} confidence) in {processing_time:.2f}s")
        
        return result
        
    except Exception as e:
        logger.error(f"Video inference failed: {str(e)}")
        logger.debug(traceback.format_exc())
        return {
            'file_path': video_path,
            'modality': 'video',
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        }

def infer_audio(model, audio_path):
    """Audio inference with comprehensive logging"""
    logger.info(f"Starting audio inference: {audio_path}")
    
    try:
        # Validate input
        validate_input_file(audio_path, ['.wav', '.mp3', '.flac', '.m4a', '.ogg'])
        
        start_time = time.time()
        
        # Convert to mel-spectrogram
        mel = audio_to_mel(audio_path)
        x = torch.tensor(mel).unsqueeze(0).unsqueeze(0).float().to(CONFIG["device"])
        logger.debug(f"Audio tensor shape: {x.shape}")
        
        # Run inference
        model.eval()
        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits, 1).cpu().numpy()[0]
        
        processing_time = time.time() - start_time
        
        result = {
            'file_path': audio_path,
            'modality': 'audio',
            'probabilities': {
                'real': float(probs[0]),
                'fake': float(probs[1])
            },
            'prediction': 'fake' if probs[1] > probs[0] else 'real',
            'confidence': float(max(probs)),
            'processing_time': processing_time,
            'timestamp': datetime.now().isoformat()
        }
        
        logger.info(f"Audio inference completed: {result['prediction']} "
                   f"({result['confidence']:.3f} confidence) in {processing_time:.3f}s")
        
        return result
        
    except Exception as e:
        logger.error(f"Audio inference failed: {str(e)}")
        logger.debug(traceback.format_exc())
        return {
            'file_path': audio_path,
            'modality': 'audio',
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        }

logger.info("Inference functions defined successfully")


INFO - Inference functions defined successfully


In [5]:
# Cell 5: Unified Inference Interface with Logging
class DeepfakeInferencePipeline:
    """Unified inference pipeline with comprehensive logging"""
    
    def __init__(self, models_dir=None):
        self.models_dir = models_dir or CONFIG["models_dir"]
        self.loaded_models = {}
        logger.info(f"Initializing DeepfakeInferencePipeline with models from: {self.models_dir}")
        
        # Discover available models
        self.discover_models()
    
    def discover_models(self):
        """Discover available model files"""
        logger.info("Discovering available models...")
        
        if not os.path.exists(self.models_dir):
            logger.error(f"Models directory not found: {self.models_dir}")
            return
        
        model_files = [f for f in os.listdir(self.models_dir) if f.endswith('.pth')]
        logger.info(f"Found {len(model_files)} model files")
        
        self.available_models = {}
        for model_file in model_files:
            # Parse model type from filename
            if 'image' in model_file.lower():
                model_type = 'image'
            elif 'video' in model_file.lower():
                model_type = 'video'
            elif 'audio' in model_file.lower():
                model_type = 'audio'
            else:
                logger.warning(f"Could not determine type for model: {model_file}")
                continue
            
            model_path = os.path.join(self.models_dir, model_file)
            self.available_models[model_type] = {
                'path': model_path,
                'filename': model_file,
                'size': os.path.getsize(model_path)
            }
            logger.debug(f"Registered {model_type} model: {model_file}")
        
        logger.info(f"Available models: {list(self.available_models.keys())}")
    
    def get_model(self, model_type):
        """Load and cache model"""
        if model_type not in self.available_models:
            raise ValueError(f"No {model_type} model available. Available: {list(self.available_models.keys())}")
        
        if model_type not in self.loaded_models:
            model_info = self.available_models[model_type]
            logger.info(f"Loading {model_type} model for first time...")
            self.loaded_models[model_type] = load_model(model_info['path'], model_type)
        else:
            logger.debug(f"Using cached {model_type} model")
        
        return self.loaded_models[model_type]
    
    def infer_single(self, file_path, model_type=None):
        """Run inference on single file"""
        logger.info(f"Single file inference request: {file_path}")
        
        # Auto-detect model type if not specified
        if model_type is None:
            model_type = self.detect_modality(file_path)
            logger.info(f"Auto-detected modality: {model_type}")
        
        # Get appropriate model
        model = self.get_model(model_type)
        
        # Run inference
        if model_type == 'image':
            return infer_image(model, file_path)
        elif model_type == 'video':
            return infer_video(model, file_path)
        elif model_type == 'audio':
            return infer_audio(model, file_path)
        else:
            raise ValueError(f"Unsupported modality: {model_type}")
    
    def infer_batch(self, file_paths, model_type=None, save_results=True):
        """Run inference on multiple files"""
        logger.info(f"Batch inference request: {len(file_paths)} files")
        start_time = time.time()
        
        results = []
        successful = 0
        failed = 0
        
        for i, file_path in enumerate(file_paths):
            logger.info(f"Processing file {i+1}/{len(file_paths)}: {file_path}")
            
            try:
                result = self.infer_single(file_path, model_type)
                if 'error' not in result:
                    successful += 1
                else:
                    failed += 1
                results.append(result)
                
            except Exception as e:
                logger.error(f"Batch inference failed for {file_path}: {str(e)}")
                failed += 1
                results.append({
                    'file_path': file_path,
                    'error': str(e),
                    'timestamp': datetime.now().isoformat()
                })
        
        total_time = time.time() - start_time
        
        batch_summary = {
            'total_files': len(file_paths),
            'successful': successful,
            'failed': failed,
            'total_time': total_time,
            'avg_time_per_file': total_time / len(file_paths),
            'timestamp': datetime.now().isoformat(),
            'results': results
        }
        
        logger.info(f"Batch inference completed: {successful}/{len(file_paths)} successful "
                   f"in {total_time:.2f}s (avg: {batch_summary['avg_time_per_file']:.3f}s/file)")
        
        # Save results if requested
        if save_results:
            self.save_results(batch_summary)
        
        return batch_summary
    
    def detect_modality(self, file_path):
        """Auto-detect file modality based on extension"""
        ext = Path(file_path).suffix.lower()
        
        if ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
            return 'image'
        elif ext in ['.mp4', '.avi', '.mov', '.mkv', '.webm']:
            return 'video'
        elif ext in ['.wav', '.mp3', '.flac', '.m4a', '.ogg']:
            return 'audio'
        else:
            raise ValueError(f"Cannot auto-detect modality for file extension: {ext}")
    
    def save_results(self, results, filename=None):
        """Save results to JSON file"""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"/kaggle/working/inference_results_{timestamp}.json"
        
        try:
            with open(filename, 'w') as f:
                json.dump(results, f, indent=2, default=str)
            logger.info(f"Results saved to: {filename}")
        except Exception as e:
            logger.error(f"Failed to save results: {str(e)}")
    
    def get_system_info(self):
        """Get system information for logging"""
        return {
            'device': CONFIG['device'],
            'available_models': list(self.available_models.keys()),
            'loaded_models': list(self.loaded_models.keys()),
            'torch_version': torch.__version__,
            'cuda_available': torch.cuda.is_available(),
            'timestamp': datetime.now().isoformat()
        }

# Initialize global pipeline instance
pipeline = DeepfakeInferencePipeline()
logger.info("DeepfakeInferencePipeline initialized successfully")


INFO - Initializing DeepfakeInferencePipeline with models from: /kaggle/input/all-models-trained
INFO - Discovering available models...
INFO - Found 5 model files
INFO - Available models: ['video', 'image', 'audio']
INFO - DeepfakeInferencePipeline initialized successfully


In [6]:
# Cell 6: Example Usage with Comprehensive Logging

# Example 1: Single file inference
def test_single_inference():
    """Test single file inference with logging"""
    logger.info("="*50)
    logger.info("TESTING SINGLE FILE INFERENCE")
    logger.info("="*50)
    
    # Example paths (replace with actual test files)
    test_files = {
         'image': '/kaggle/input/sample-media/IMG-20250713-WA0027.jpg',
         'video': '/kaggle/input/sample-media/Ayush - Slide 6.mp4',
         'audio': '/kaggle/input/sample-media/file_example_WAV_2MG.wav'
    }
    
    for modality, file_path in test_files.items():
        if os.path.exists(file_path):
            logger.info(f"Testing {modality} inference...")
            try:
                result = pipeline.infer_single(file_path)
                logger.info(f"Result: {json.dumps(result, indent=2)}")
            except Exception as e:
                logger.error(f"Test failed for {modality}: {str(e)}")
        else:
            logger.warning(f"Test file not found: {file_path}")

# Example 2: Batch inference
def test_batch_inference():
    """Test batch inference with logging"""
    logger.info("="*50)
    logger.info("TESTING BATCH INFERENCE")
    logger.info("="*50)
    
    # Example batch (replace with actual file paths)
    test_batch = [
         '/kaggle/input/sample-media/IMG-20250713-WA0027.jpg',
         '/kaggle/input/sample-media/Ayush - Slide 6.mp4'
    ]
    
    if test_batch:
        try:
            results = pipeline.infer_batch(test_batch, save_results=True)
            logger.info(f"Batch results summary: {results['successful']}/{results['total_files']} successful")
        except Exception as e:
            logger.error(f"Batch test failed: {str(e)}")
    else:
        logger.info("No test files specified for batch inference")

# Example 3: Command-line interface
def run_cli():
    """Command-line interface with proper logging"""
    logger.info("="*50)
    logger.info("COMMAND-LINE INTERFACE")
    logger.info("="*50)
    
    # Only run CLI in script mode, not in Jupyter
    if __name__ == "__main__" and not hasattr(sys, "ps1"):
        try:
            parser = argparse.ArgumentParser(
                description="Deepfake Detection Inference Pipeline",
                formatter_class=argparse.RawDescriptionHelpFormatter,
                epilog="""
Examples:
  python inference.py single image /path/to/image.jpg
  python inference.py single auto /path/to/file.mp4
  python inference.py batch /path/to/folder/
                """
            )
            
            parser.add_argument("mode", choices=["single", "batch"], 
                              help="Inference mode")
            parser.add_argument("modality", nargs='?', 
                              choices=["image", "video", "audio", "auto"], 
                              default="auto",
                              help="Input modality (auto-detect if not specified)")
            parser.add_argument("input", help="Input file or folder path")
            parser.add_argument("--output", help="Output file path for results")
            parser.add_argument("--log-level", default="INFO",
                              choices=["DEBUG", "INFO", "WARNING", "ERROR"],
                              help="Logging level")
            
            args, extra = parser.parse_known_args()
            
            # Update logging level
            if args.log_level != "INFO":
                logging.getLogger().setLevel(getattr(logging, args.log_level))
                logger.info(f"Log level changed to: {args.log_level}")
            
            logger.info(f"CLI Arguments: {vars(args)}")
            
            if args.mode == "single":
                result = pipeline.infer_single(args.input, 
                                             None if args.modality == "auto" else args.modality)
                print(json.dumps(result, indent=2))
                
                if args.output:
                    pipeline.save_results(result, args.output)
                    
            elif args.mode == "batch":
                # Collect files from directory
                if os.path.isdir(args.input):
                    files = []
                    for ext in ['jpg', 'jpeg', 'png', 'mp4', 'avi', 'wav', 'mp3']:
                        files.extend(Path(args.input).glob(f"*.{ext}"))
                    file_paths = [str(f) for f in files]
                else:
                    file_paths = [args.input]
                
                logger.info(f"Found {len(file_paths)} files for batch processing")
                
                results = pipeline.infer_batch(file_paths,
                                             None if args.modality == "auto" else args.modality,
                                             save_results=True)
                
                print(f"Batch completed: {results['successful']}/{results['total_files']} successful")
                
                if args.output:
                    pipeline.save_results(results, args.output)
                    
        except Exception as e:
            logger.error(f"CLI execution failed: {str(e)}")
            logger.debug(traceback.format_exc())
            sys.exit(1)
    else:
        logger.info("Notebook environment - CLI skipped")

# System information logging
logger.info("="*50)
logger.info("SYSTEM INFORMATION")
logger.info("="*50)
system_info = pipeline.get_system_info()
for key, value in system_info.items():
    logger.info(f"{key}: {value}")

# Run tests (uncomment to execute)
test_single_inference()
test_batch_inference()
run_cli()

logger.info("="*50)
logger.info("INFERENCE PIPELINE READY")
logger.info("="*50)
logger.info(f"Available models: {list(pipeline.available_models.keys())}")
logger.info(f"Log file: {LOG_FILE}")
logger.info("Use pipeline.infer_single() or pipeline.infer_batch() for inference")


INFO - ==================================================
INFO - SYSTEM INFORMATION
INFO - ==================================================
INFO - device: cuda
INFO - available_models: ['video', 'image', 'audio']
INFO - loaded_models: []
INFO - torch_version: 2.6.0+cu124
INFO - cuda_available: True
INFO - timestamp: 2025-07-20T09:35:46.666091
INFO - ==================================================
INFO - TESTING SINGLE FILE INFERENCE
INFO - ==================================================
INFO - Testing image inference...
INFO - Single file inference request: /kaggle/input/sample-media/IMG-20250713-WA0027.jpg
INFO - Auto-detected modality: image
INFO - Loading image model for first time...
INFO - Loading image model from: /kaggle/input/all-models-trained/celebdf_v2_image_best.pth


Downloading: "https://github.com/pytorch/vision/zipball/v0.13.1" to /root/.cache/torch/hub/v0.13.1.zip
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


INFO - Loaded image model in 3.34s (strict=False)
INFO - Starting image inference: /kaggle/input/sample-media/IMG-20250713-WA0027.jpg
INFO - Input file validated: /kaggle/input/sample-media/IMG-20250713-WA0027.jpg (110722 bytes, .jpg)
INFO - Image inference completed: fake (0.542 confidence) in 1.728s
INFO - Result: {
  "file_path": "/kaggle/input/sample-media/IMG-20250713-WA0027.jpg",
  "modality": "image",
  "probabilities": {
    "real": 0.45793306827545166,
    "fake": 0.5420669317245483
  },
  "prediction": "fake",
  "confidence": 0.5420669317245483,
  "processing_time": 1.727529764175415,
  "timestamp": "2025-07-20T09:35:51.751952"
}
INFO - Testing video inference...
INFO - Single file inference request: /kaggle/input/sample-media/Ayush - Slide 6.mp4
INFO - Auto-detected modality: video
INFO - Loading video model for first time...
INFO - Loading video model from: /kaggle/input/all-models-trained/ff_c23_video_best.pth
INFO - Loaded video model in 1.28s (strict=False)
INFO - Starti

In [7]:
# Cell 7: Usage Examples and Documentation

print("""
=============================================================================
DEEPFAKE DETECTION INFERENCE PIPELINE - USAGE DOCUMENTATION
=============================================================================

FEATURES:
- Comprehensive logging to file and console
- Support for image, video, and audio inference
- Automatic model loading and caching
- Batch processing capabilities
- Error handling and recovery
- Detailed performance metrics
- JSON output format

USAGE EXAMPLES:

1. Single File Inference:
   result = pipeline.infer_single('/path/to/image.jpg')
   result = pipeline.infer_single('/path/to/video.mp4', 'video')
   
2. Batch Inference:
   files = ['/path/to/file1.jpg', '/path/to/file2.mp4']
   results = pipeline.infer_batch(files, save_results=True)
   
3. Auto-detect Modality:
   result = pipeline.infer_single('/path/to/unknown_file.ext')
   
4. Save Results:
   pipeline.save_results(result, '/path/to/output.json')

OUTPUT FORMAT:
{
  "file_path": "/path/to/file.jpg",
  "modality": "image",
  "probabilities": {
    "real": 0.234,
    "fake": 0.766
  },
  "prediction": "fake",
  "confidence": 0.766,
  "processing_time": 0.123,
  "timestamp": "2025-07-20T13:54:00"
}

LOG FILES:
- Main log: /kaggle/working/logs/deepfake_inference_YYYYMMDD_HHMMSS.log
- Results: /kaggle/working/inference_results_YYYYMMDD_HHMMSS.json

AVAILABLE MODELS:
""")

for model_type, info in pipeline.available_models.items():
    print(f"- {model_type.upper()}: {info['filename']} ({info['size']/1024/1024:.1f} MB)")

print(f"""
SYSTEM STATUS:
- Device: {CONFIG['device']}
- Models Directory: {CONFIG['models_dir']}
- Log Level: {logging.getLogger().level}
- PyTorch Version: {torch.__version__}

Ready for inference! 🚀
""")



DEEPFAKE DETECTION INFERENCE PIPELINE - USAGE DOCUMENTATION

FEATURES:
- Comprehensive logging to file and console
- Support for image, video, and audio inference
- Automatic model loading and caching
- Batch processing capabilities
- Error handling and recovery
- Detailed performance metrics
- JSON output format

USAGE EXAMPLES:

1. Single File Inference:
   result = pipeline.infer_single('/path/to/image.jpg')
   result = pipeline.infer_single('/path/to/video.mp4', 'video')
   
2. Batch Inference:
   files = ['/path/to/file1.jpg', '/path/to/file2.mp4']
   results = pipeline.infer_batch(files, save_results=True)
   
3. Auto-detect Modality:
   result = pipeline.infer_single('/path/to/unknown_file.ext')
   
4. Save Results:
   pipeline.save_results(result, '/path/to/output.json')

OUTPUT FORMAT:
{
  "file_path": "/path/to/file.jpg",
  "modality": "image",
  "probabilities": {
    "real": 0.234,
    "fake": 0.766
  },
  "prediction": "fake",
  "confidence": 0.766,
  "processing_time": 0